In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv (r'Capstone_data_Jan19_Nov19.csv')  # 181559 rows × 642 columns
readme = pd.read_csv(r'data_type.csv')

C:\Users\shaow\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (59,66,85,95,512,513,514,515,516,517,518,519,520,521,522,529,531,536,539) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
readme_type = readme[['Column Name', 'Type']]
readme_numerical = readme_type[readme_type['Type'] == 'Numerical']
readme_categorical = readme_type[readme_type['Type'] == 'Categorical']
num_data = data._get_numeric_data()
num_cols = list(num_data.columns)
removed_vars = {'X640', 'X641', 'X642','X101'} # X640, x641, x642, X101 are not numericals, excluded
num_cols = [ele for ele in num_cols if ele not in removed_vars]

var_cols = [ele for ele in data.columns if ele not in removed_vars]
cat_cols = list(set(data.columns) - set(num_cols) )

readme_num_list = readme_numerical['Column Name'].to_list()

# check difference between two lists: Find out which columns are marked as numerical variables but should be categorical
cat_marked_as_num = list(set(num_cols) - set(readme_num_list))
cat_marked_as_num.sort()

data[cat_marked_as_num] = data[cat_marked_as_num].astype('category')

**Starting from here, the code is different from other notebooks**

In [4]:
text_variables = data.select_dtypes(include='object')  # we assume all object columns are text columns
text_vars = text_variables.columns.values.tolist()
text_vars.remove('X101')  # we remove X101 from the text columns since it is an ID feature
data_text = data.loc[:, text_vars]  # pick text columns from the original dataset
# data_text = pd.concat([data_text, data['X642']], axis = 1)

In [5]:
na_unknown_combine = []
disease_col = []
onehot_with_na = []
onehot_without_na = []

# Process X87: In this column we have 'positive', 'negative', '1' and '0'; we change 1 to 'positive', 0 to 'negative'
data_text.loc[(data_text.X87 == '0001.000'),'X87'] = 'POSITIVE OR REACTIVE'
data_text.loc[(data_text.X87 == '0000.000'),'X87'] = 'NEGATIVE'

# X496-X510, X521-X526, X528-X536 are columns with 'yes','no','unknown' and na, we want to combine na with 'unknown' and together form the 'nan_unknown' category
for i in range(496, 511):
    na_unknown_combine.append('X' + str(i))
for i in range(521, 527):
    na_unknown_combine.append('X' + str(i))
for i in range(528, 537):
    na_unknown_combine.append('X' + str(i))
for ele in na_unknown_combine:
    data_text[ele] = data_text[ele].where(data_text[ele]!='UNKNOWN','na_or_unknown')
    data_text[ele] = data_text[ele].where(data_text[ele].isna()==False,'na_or_unknown')
'''
# double check if successfully converted na and 'unknown' to 'na_or_unknown'
for ele in na_unknown_combine:
    print(data_text[ele].unique())
'''
for j in range(538, 564):  # X538-X563 are columns contain diseases, the SPECIAL columns!!!
    disease_col.append('X' + str(j))

# we include columns that could be easily one hot encoded into the one_hot_col vector
one_hot_col = [ele for ele in text_vars if ele not in disease_col]  # as long as it is not a disease column, it is one hot encoded

# when we use pd.get_dummies, we need to separate columns that contain na values from columns that do not contain na values
# therefore creat two lists, one stores column(with na values) ids, one stores column(without na values) ids
for ele in one_hot_col:
    if data_text[ele].isna().values.any():
        onehot_with_na.append(ele)
    else:
        onehot_without_na.append(ele)

In [6]:
df1 = pd.get_dummies(data_text[onehot_with_na], dummy_na = True)  # one hot encode text columns with na value
df2 = pd.get_dummies(data_text[onehot_without_na])  # one hot encode text columns without na value

In [14]:
non_text = [ele for ele in data.columns.values.tolist() if ele not in text_vars]

In [15]:
concat_data = pd.concat([data[non_text], df1, df2], axis = 1)
concat_data  # all data except for 'disease' columns

,X640,X641,X642,X1,X2,X3,X4,X5,X6,X7,...,X533_na_or_unknown,X534_NO,X534_YES,X534_na_or_unknown,X535_NO,X535_YES,X535_na_or_unknown,X536_NO,X536_YES,X536_na_or_unknown
0,1,101,0,1.272695,-0.309136,1961,-0.279328,43.891779,157.48,133.000000,...,1,0,0,1,1,0,0,1,0,0
1,2,102,1,1.194891,-0.309136,1962,-0.279328,29.697368,193.04,116.666667,...,1,0,0,1,1,0,0,1,0,0
2,3,103,1,1.817320,-0.309136,1953,-0.279328,29.598352,170.18,125.000000,...,1,0,0,1,1,0,0,1,0,0
3,4,104,0,2.361946,-0.344643,1947,-0.316563,31.321007,170.18,144.666667,...,1,0,0,1,1,0,0,1,0,0
4,5,105,0,1.428302,-0.356715,1959,-0.329222,37.271479,165.10,NaN,...,1,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181554,229873,229524,0,1.194891,-0.344643,1962,-0.316563,31.635000,152.40,NaN,...,1,0,0,1,1,0,0,1,0,0
181555,229874,229525,0,0.961480,-0.238121,1965,-0.279328,31.563265,177.80,121.333333,...,1,0,0,1,1,0,0,1,0,0
181556,229875,229526,1,1.350498,-0.362396,1960,-0.335180,31.946982,165.10,NaN,...,1,0,0,1,1,0,0,1,0,0
181557,229876,229527,1,1.039284,-0.309136,1964,-0.279328,28.188906,170.18,110.666667,...,1,0,0,1,1,0,0,1,0,0


**这部分往下暂时先别看，是处理disease columns的code**

In [7]:
# Process disease columns
tmp_disease_df = data_text[disease_col]
tmp_disease_df

,X538,X539,X540,X541,X542,X543,X544,X545,X546,X547,...,X554,X555,X556,X557,X558,X559,X560,X561,X562,X563
0,HIGHBLOODP,HIGHBLOODP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA|NA|NA,IRRPULSE|IRRPULSE|IRRPULSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANCER
2,HIGHBLOODP,HIGHBLOODP,DIABETESDM|DIABETESDM|DIABETESDM|DIABETESDM|DI...,DIABETES|DIABETES|DIABETES|DIABETES|DIABETES|D...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mammogram,NaN,NaN
3,HIGHBLOODP,HIGHBLOODP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,DIVERTCLTS,INTDSRDR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,DIABETESDM|DIABETESDM|DIABETESDM|DIABETESDM|DI...,DIABETES|DIABETES|DIABETES|DIABETES|DIABETES|D...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,CHRNICPAIN,CHRNICPAIN,NaN,NaN,Spinal Cord Stimulator,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ARTHRITIS2|ARTHRITIS2,ARTHRITIS2|ARTHRITIS2,DEPRESSION,OTHPSYMHD,NaN,NaN,NaN,HRTDSRDR
181555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HRTDSRDR
181556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
for ele in tmp_disease_df:
    test = set()
    dp=tmp_disease_df[ele].str.split('|').dropna()
    for i in dp:
        col = set(i)
        for _ in col:
            if _ not in test:
                test.add(_)
    print(ele, ':', len(test))

X538 : 54
X539 : 13
X540 : 15
X541 : 5
X542 : 38
X543 : 8
X544 : 65
X545 : 7
X546 : 62
X547 : 11
X548 : 75
X549 : 11
X550 : 18
X551 : 4
X552 : 37
X553 : 6
X554 : 53
X555 : 8
X556 : 93
X557 : 8
X558 : 37
X559 : 3
X560 : 4160
X561 : 479
X562 : 1
X563 : 4


In [74]:
# from X538-X563, all the disease types are stored in the total_disease set
total_disease = set()
for ele in tmp_disease_df:
    df = tmp_disease_df[ele].str.split('|').dropna()
    for i in df:
        col = set(i)
        for _ in col:
            if _ not in total_disease:
                total_disease.add(_)

In [79]:
total_disease

{'Unknown - Testing for genetics, blood type, hepatitis, whooping cough, and HIV',
 'Bladder Sling Removal',
 'Tummy tuck,breast lift and reduction',
 'Repair a torn meniscus.',
 'Other - Electrophysiological study of the heart',
 'Gastric Sleeve',
 'Correct obstruction left side of nose',
 'Recurring Dislocation Of Right Shoulder Surgery',
 'nasal cleaning',
 'foot surgery on left foot',
 'Right ankle othoplasty',
 'Other - BMI Check',
 'Eye surgery for detached retina',
 'Knee Surgery',
 'Infertlity',
 'Removed small lipoma (fatty tissue) off of forhead',
 'Remove the blood around the liver',
 'JOINTPAIN',
 'Broken Ankle',
 'Stitch on arm wound',
 'Colon Resection',
 'Uterine Fibroid Removed',
 'ankle kidner procedure',
 'Surgery remove growth on the outside of the cheek',
 'Root Canal and Bridge',
 'Torn rotator cuff surgery',
 'Orbital Eye Socket Surgery',
 'Collarbone',
 'Rotator Cuff Repair Surgery',
 'Spinal Cord Stimulation System (Implant)',
 'OTHER3K',
 'Sebaceous cyst remova

In [ ]:
dic = {}
for i in range(tmp_disease_df.shape[0]):
    df = tmp_disease_df.loc[i, :]
    print(df)

<font size="5">Statistical Significance (没用到，问题不大)</font>

In [52]:
from scipy import stats
na_col = data_text.columns[data_text.isna().any()].tolist()

c=0
notrandom=[]
for i, col in enumerate(data_text.columns[data_text.isna().any()].tolist()):
    missing_y = data_text[data_text[col].isna()==True]['X642'].tolist()
    non_missing_y = data_text[data_text[col].isna()==False]['X642'].tolist()
    t_check=stats.ttest_ind(missing_y,non_missing_y)
    alpha=0.005    
    if(t_check[1]<alpha):
        c+=1
        notrandom.append([col,str(data_text[col].dtype),t_check[1]])
        print(c, col, t_check)

1 X23 Ttest_indResult(statistic=-15.199934319613416, pvalue=3.811914776919284e-52)
2 X50 Ttest_indResult(statistic=-4.581024915554961, pvalue=4.6300966486671795e-06)
3 X59 Ttest_indResult(statistic=8.17671447629593, pvalue=2.9354226088076046e-16)
4 X71 Ttest_indResult(statistic=-4.425912824562112, pvalue=9.60911464645856e-06)
5 X81 Ttest_indResult(statistic=-2.8710131591686814, pvalue=0.0040920578861814525)
6 X82 Ttest_indResult(statistic=-13.436928850315883, pvalue=3.844147524036602e-41)
7 X87 Ttest_indResult(statistic=31.424959175754513, pvalue=3.527724133159656e-216)
8 X496 Ttest_indResult(statistic=5.5262347304964425, pvalue=3.276239206576001e-08)
9 X521 Ttest_indResult(statistic=23.50552437429395, pvalue=5.456063730596919e-122)
10 X522 Ttest_indResult(statistic=23.502421741379056, pvalue=5.868280810070934e-122)
11 X523 Ttest_indResult(statistic=23.502421741379056, pvalue=5.868280810070934e-122)
12 X524 Ttest_indResult(statistic=23.502421741379056, pvalue=5.868280810070934e-122)
13

In [50]:
# check X496-X510 X528-X536 unknown and na
unknown_col = []
for i in range(496, 511):
    unknown_col.append('X'+str(i))
for i in range(528, 537):
    unknown_col.append('X'+str(i))
unknown_text = data.loc[:,unknown_col]  # pick text columns
unknown_text = pd.concat([unknown_text, data['X642']], axis = 1)

c1=0
notrandom1=[]
for i, col in enumerate(unknown_text.columns[unknown_text.isna().any()].tolist()):
    missing_y_na = unknown_text[unknown_text[col].isna()==True]['X642'].tolist()
    missing_y_unknown = unknown_text[unknown_text[col]=='UNKNOWN']['X642'].tolist()
    t_check=stats.ttest_ind(missing_y,missing_y_unknown)
    alpha=0.005    
    if(t_check[1]<alpha):
        c1+=1
        notrandom1.append([col,str(unknown_text[col].dtype),t_check[1]])
        print(c, col, t_check)

40 X496 Ttest_indResult(statistic=4.5347447171752036, pvalue=5.771923072764068e-06)
40 X533 Ttest_indResult(statistic=4.5347447171752036, pvalue=5.771923072764068e-06)
